# Introduction
**This will be your workspace for the [Machine Learning course](https://www.kaggle.com/learn/machine-learning).**

You will need to translate the concepts to work with the data in this notebook, the Iowa data. Each page in the Machine Learning course includes instructions for what code to write at that step in the course.

# Write Your Code Below

In [ ]:
import pandas as pd

main_file_path = '../input/house-prices-advanced-regression-techniques/train.csv' # this is the path to the Iowa data that you will use
data = pd.read_csv(main_file_path)

print(data.describe())
print(data.head())
print(data.columns)

In [ ]:
print(data["SalePrice"].describe())
print(data["SalePrice"].head())

In [ ]:
two_columns = data[["BldgType","SalePrice"]]
two_columns.describe()

In [ ]:
feature_list = ["LotArea","YearBuilt","1stFlrSF","2ndFlrSF","FullBath","BedroomAbvGr","TotRmsAbvGrd"]
X = data[feature_list]
y = data["SalePrice"]
X.head()
y.head()

In [ ]:
from sklearn.tree import DecisionTreeRegressor as dt

iowa_model = dt()
iowa_model.fit(X,y)

In [ ]:
iowa_model.predict(X.head())

In [ ]:
from sklearn.metrics import mean_absolute_error

predicted_home_prices = iowa_model.predict(X)
mean_absolute_error(y, predicted_home_prices)

In [ ]:
#split training and validation data using scikit-learns inbuilt function

from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(X, y,random_state = 0)

iowa_model.fit(train_X,train_y)
predicted_home_prices = iowa_model.predict(val_X)
mean_absolute_error(val_y,predicted_home_prices)

In [ ]:
def get_mae(max_leaf_nodes, predictors_train, predictors_val, targ_train, targ_val):
    model = dt(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(predictors_train, targ_train)
    preds_val = model.predict(predictors_val)
    mae = mean_absolute_error(targ_val, preds_val)
    return(mae)

for max_leaf_nodes in [5, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae))

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest_model = RandomForestRegressor()
forest_model.fit(train_X, train_y)
melb_preds = forest_model.predict(val_X)
print(mean_absolute_error(val_y, melb_preds))

In [ ]:
test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')
test_features = test[feature_list]
predicted_prices = forest_model.predict(test_features)
print(predicted_prices)

In [ ]:
#creat submission file called submission.csv
my_submission = pd.DataFrame({'Id': test.Id, 'SalePrice': predicted_prices})
my_submission.to_csv('submission.csv', index=False)


**If you have any questions or hit any problems, come to the [Learn Discussion](https://www.kaggle.com/learn-forum) for help. **

**Return to [ML Course Index](https://www.kaggle.com/learn/machine-learning)**